In [0]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path='/content/drive/My Drive/tushar/aug/'
folders=os.listdir(path)

In [27]:
folders

['1', '0']

In [28]:
images=[]
Y=[]
cls=0
for folder in folders:
  files=os.listdir(path+folder)
  Y.extend([cls for _ in range(len(files))])
  cls+=1
  for file in files:
    print(path+folder+file)
    img=cv2.imread(path+folder+'//'+file)
    img=cv2.resize(img,(60,60),interpolation=cv2.INTER_AREA)
    #img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)            
    images.append(img)

/content/drive/My Drive/tushar/aug/1aug_0_1674.jpg
/content/drive/My Drive/tushar/aug/1aug_0_8574.jpg
/content/drive/My Drive/tushar/aug/1aug_0_7403.jpg
/content/drive/My Drive/tushar/aug/1aug_0_7743.jpg
/content/drive/My Drive/tushar/aug/1aug_0_8818.jpg
/content/drive/My Drive/tushar/aug/1aug_0_557.jpg
/content/drive/My Drive/tushar/aug/1aug_0_1008.jpg
/content/drive/My Drive/tushar/aug/1aug_0_7132.jpg
/content/drive/My Drive/tushar/aug/1aug_0_294.jpg
/content/drive/My Drive/tushar/aug/1aug_0_6590.jpg
/content/drive/My Drive/tushar/aug/1aug_0_2938.jpg
/content/drive/My Drive/tushar/aug/1aug_0_5008.jpg
/content/drive/My Drive/tushar/aug/1aug_0_9706.jpg
/content/drive/My Drive/tushar/aug/1aug_0_3839.jpg
/content/drive/My Drive/tushar/aug/1aug_0_9640.jpg
/content/drive/My Drive/tushar/aug/1aug_0_842.jpg
/content/drive/My Drive/tushar/aug/1aug_0_9760.jpg
/content/drive/My Drive/tushar/aug/1aug_0_5129.jpg
/content/drive/My Drive/tushar/aug/1aug_0_154.jpg
/content/drive/My Drive/tushar/aug/

In [0]:
X=np.array(images)
Y=np.array(Y[:])

In [30]:
import pandas as pd
pd.Series(Y).value_counts()

0    201
1    199
dtype: int64

In [0]:
from sklearn.model_selection import train_test_split

xtr,xte,ytr,yte=train_test_split(images,Y,test_size=0.2)

In [0]:
xtr=np.array(xtr)
ytr=np.array(ytr)
xte=np.array(xte)
yte=np.array(yte)

In [33]:
xtr.shape,ytr.shape,xte.shape,yte.shape

((320, 60, 60, 3), (320,), (80, 60, 60, 3), (80,))

In [0]:
xtr=xtr/255.0
xte=xte/255.0
from keras.utils import to_categorical
ytr=to_categorical(ytr)
yte=to_categorical(yte)

In [35]:
xtr.shape,ytr.shape,xte.shape,yte.shape

((320, 60, 60, 3), (320, 2), (80, 60, 60, 3), (80, 2))

In [0]:
image_size=xtr[0].shape[0]
opt_size=ytr[0].shape[0]

In [0]:
xtr=xtr.reshape(xtr.shape[0],60,60,3)
xte=xte.reshape(xte.shape[0],60,60,3)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Flatten,MaxPool2D

model=Sequential()

model.add(Conv2D(16,input_shape=(image_size,image_size,3),kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=1))

model.add(Conv2D(32,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))

model.add(Conv2D(64,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))

model.add(Conv2D(128,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=1))

model.add(Conv2D(256,kernel_size=1,activation='tanh'))             ##new layer

model.add(Flatten())
model.add(Dense(26,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(opt_size,activation='softmax'))

In [39]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(xtr,ytr,epochs=10,validation_data=(xte,yte))

Epoch 1/10
10/10 [==============================] - 3s 299ms/step - loss: 0.7455 - accuracy: 0.5094 - val_loss: 0.6992 - val_accuracy: 0.4750
Epoch 2/10
10/10 [==============================] - 3s 285ms/step - loss: 0.6748 - accuracy: 0.5031 - val_loss: 0.6012 - val_accuracy: 0.4750
Epoch 3/10
10/10 [==============================] - 3s 284ms/step - loss: 0.4547 - accuracy: 0.5031 - val_loss: 0.3594 - val_accuracy: 0.4750
Epoch 4/10
10/10 [==============================] - 3s 288ms/step - loss: 0.2866 - accuracy: 0.9312 - val_loss: 0.1024 - val_accuracy: 1.0000
Epoch 5/10
10/10 [==============================] - 3s 285ms/step - loss: 0.0176 - accuracy: 1.0000 - val_loss: 5.6003e-07 - val_accuracy: 1.0000
Epoch 6/10
10/10 [==============================] - 3s 289ms/step - loss: 0.5872 - accuracy: 0.9438 - val_loss: 1.9371e-07 - val_accuracy: 1.0000
Epoch 7/10
10/10 [==============================] - 3s 285ms/step - loss: 0.0940 - accuracy: 0.9875 - val_loss: 0.0058 - val_accuracy: 1.000

In [0]:
#done